# Journal 05: DateTime Aggregations

As the title suggests, today is about working with datetimes and trying to aggregate data based on that value. This is perhaps more important in the long-run, but for the moment it will be useful to get that `'date'` column in a usable format. Plus it is a good start in beginning to work with this data in *Time-Series* analytical manner (more skills, more analytics, more data, more beter).

So, the rough order of this journal is likely to be as the following:
1. Modify `read_clean_data()` to properly convert the `'date'` column into a `datetime` format.
2. Track percentage changes over the following time intervals:
    1. Day to Day
    2. Week to week
    3. Month to Month
    4. Quarterly
    5. Year to Year

## Utilizing DateTimes

Returning to our `read_clean_data()` function:

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def read_clean_data(file, ochl_only=False):
    file_path = 'data/' + file

    data = pd.read_csv(file_path)

    data = data.loc[::-1].reset_index(drop=True)

    filename = file.split('_')
    stock_label = filename[0]

    data.insert(0, 'Label', value=stock_label)
    
    vol = data.pop('Volume')
    data.insert(2, 'Volume', vol)

    ochl_columns = ['Open', 'Close/Last', 'High', 'Low']

    for column in ochl_columns:
        data[column] = data[column].str.replace('$', '').astype('float64')

    if ochl_only:
        return data[ochl_columns]

    return data

aapl = read_clean_data('AAPL_historical.csv')

aapl.head()


,Label,Date,Volume,Close/Last,Open,High,Low
0,AAPL,09/30/2013,259059760,17.0268,17.0446,17.2021,16.9433
1,AAPL,10/01/2013,352776899,17.4271,17.0875,17.4693,17.0850
2,AAPL,10/02/2013,287196445,17.4843,17.3439,17.5643,17.2768
3,AAPL,10/03/2013,315209314,17.2646,17.5182,17.5839,17.1692
4,AAPL,10/04/2013,258552177,17.2511,17.2807,17.3071,17.0929


Now let's check the type of the `Date` column

In [4]:
type(aapl['Date'][0])

str

So it's a string, which is to be expected, now we need to properly convert this into a datetime object, one that we can use for analytical purposes.

In [5]:
test = pd.to_datetime(aapl['Date'])

test.head()

0   2013-09-30
1   2013-10-01
2   2013-10-02
3   2013-10-03
4   2013-10-04
Name: Date, dtype: datetime64[ns]

In [6]:
type(test[0])

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
aapl['Date'] = pd.to_datetime(aapl['Date'])

type(aapl['Date'][0])

pandas._libs.tslibs.timestamps.Timestamp

Perfect! Now that is out of the way, let's see what we can do with it!

First thing that comes to mind in working with datetimes is subsetting a set of data in a time range, let's get our bearings by pulling a year's worth of data:

In [58]:
# year_mask = (aapl['Date'] >= '2014-01-01') & (aapl['Date'] <= '2014-12-31')
# year_mask

year_mask = (aapl['Date'] >= '2014-01') & (aapl['Date'] <= '2014-03')
year_mask

0       False
1       False
2       False
3       False
4       False
        ...  
2511    False
2512    False
2513    False
2514    False
2515    False
Name: Date, Length: 2516, dtype: bool

In [59]:
data_2014 = aapl[year_mask]

for ind in data_2014.index:
    print(data_2014['Date'])

65    2014-01-02
66    2014-01-03
67    2014-01-06
68    2014-01-07
69    2014-01-08
70    2014-01-09
71    2014-01-10
72    2014-01-13
73    2014-01-14
74    2014-01-15
75    2014-01-16
76    2014-01-17
77    2014-01-21
78    2014-01-22
79    2014-01-23
80    2014-01-24
81    2014-01-27
82    2014-01-28
83    2014-01-29
84    2014-01-30
85    2014-01-31
86    2014-02-03
87    2014-02-04
88    2014-02-05
89    2014-02-06
90    2014-02-07
91    2014-02-10
92    2014-02-11
93    2014-02-12
94    2014-02-13
95    2014-02-14
96    2014-02-18
97    2014-02-19
98    2014-02-20
99    2014-02-21
100   2014-02-24
101   2014-02-25
102   2014-02-26
103   2014-02-27
104   2014-02-28
Name: Date, dtype: datetime64[ns]
65    2014-01-02
66    2014-01-03
67    2014-01-06
68    2014-01-07
69    2014-01-08
70    2014-01-09
71    2014-01-10
72    2014-01-13
73    2014-01-14
74    2014-01-15
75    2014-01-16
76    2014-01-17
77    2014-01-21
78    2014-01-22
79    2014-01-23
80    2014-01-24
81    2014-01-

In [62]:
pd.date_range('2014-01-01', '2014-02', freq='W')

DatetimeIndex(['2014-01-05', '2014-01-12', '2014-01-19', '2014-01-26'], dtype='datetime64[ns]', freq='W-SUN')

In [72]:
from stock_analysis import extract_ochl

p_changes = [0]

aapl_ochl = extract_ochl(aapl)

aapl['avg_all'] = aapl_ochl.mean(axis=1)

aapl.head()

for x in range(1, len(aapl)):
    init = aapl.loc[x, ('avg_all')]
    prev_val = aapl.loc[x-1, ('avg_all')]

    p_change = (init - prev_val) / prev_val * 100
    p_changes.append(p_change)

aapl['p_change'] = p_changes

aapl.head()

,Label,Date,Volume,Close/Last,Open,High,Low,avg_all,p_change
0,AAPL,2013-09-30,259059760,17.0268,17.0446,17.2021,16.9433,17.054200,0.000000
1,AAPL,2013-10-01,352776899,17.4271,17.0875,17.4693,17.0850,17.267225,1.249106
2,AAPL,2013-10-02,287196445,17.4843,17.3439,17.5643,17.2768,17.417325,0.869277
3,AAPL,2013-10-03,315209314,17.2646,17.5182,17.5839,17.1692,17.383975,-0.191476
4,AAPL,2013-10-04,258552177,17.2511,17.2807,17.3071,17.0929,17.232950,-0.868760


In [75]:
data_2014 = aapl[year_mask]

data_2014['p_change'].sum()

-5.053972700957063

In [79]:
aapl['p_change'].rolling(2).sum()

0            NaN
1       1.249106
2       2.118383
3       0.677801
4      -1.060236
          ...   
2511   -2.069857
2512   -1.299543
2513    0.394730
2514   -0.990599
2515   -2.331778
Name: p_change, Length: 2516, dtype: float64

Ok, so now we have this %change between global averages of each day in regards to the previous day.

To make things easier (I hope), I'm going to extract this new value, as well as the dates, so I can use the dates as the indext, making `pd.date_range` actually usable.

In [120]:
changes = aapl[['Date', 'p_change']]

changes.head()

,Date,p_change
0,2013-09-30,0.000000
1,2013-10-01,1.249106
2,2013-10-02,0.869277
3,2013-10-03,-0.191476
4,2013-10-04,-0.868760


In [280]:
freq = pd.offsets.CustomBusinessDay(weekmask='Mon')
full_dates = pd.date_range(start=aapl.loc[0, 'Date'], end=aapl.loc[len(aapl)-1 ,'Date'], freq='2d')
len(full_dates)

print(full_dates)

dates = aapl['Date']

print(type(dates))

print(type(full_dates))

x_dates = np.asarray(dates)
x_full = np.asarray(full_dates)

print(x_full)

DatetimeIndex(['2013-09-30', '2013-10-02', '2013-10-04', '2013-10-06',
               '2013-10-08', '2013-10-10', '2013-10-12', '2013-10-14',
               '2013-10-16', '2013-10-18',
               ...
               '2023-09-08', '2023-09-10', '2023-09-12', '2023-09-14',
               '2023-09-16', '2023-09-18', '2023-09-20', '2023-09-22',
               '2023-09-24', '2023-09-26'],
              dtype='datetime64[ns]', length=1825, freq='2D')
<class 'pandas.core.series.Series'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
['2013-09-30T00:00:00.000000000' '2013-10-02T00:00:00.000000000'
 '2013-10-04T00:00:00.000000000' ... '2023-09-22T00:00:00.000000000'
 '2023-09-24T00:00:00.000000000' '2023-09-26T00:00:00.000000000']


In [281]:
# print(x_dates)
# print(type(x_dates))
# print(type(x_dates[0]))
# print(len(x_dates))
# print(x_full)
# print(type(x_full))
# print(type(x_full[0]))
# print(len(x_full))

match = np.intersect1d(x_dates, x_full)

# print(match.shape)
print('match', match)
print(type(match))

match ['2013-09-30T00:00:00.000000000' '2013-10-02T00:00:00.000000000'
 '2013-10-04T00:00:00.000000000' ... '2023-09-20T00:00:00.000000000'
 '2023-09-22T00:00:00.000000000' '2023-09-26T00:00:00.000000000']
<class 'numpy.ndarray'>


In [282]:
subset_dates = np.isin(x_dates, match)
len(subset_dates)

mondays = aapl[subset_dates]
mondays

,Label,Date,Volume,Close/Last,Open,High,Low,avg_all,p_change
0,AAPL,2013-09-30,259059760,17.0268,17.0446,17.2021,16.9433,17.054200,0.000000
2,AAPL,2013-10-02,287196445,17.4843,17.3439,17.5643,17.2768,17.417325,0.869277
4,AAPL,2013-10-04,258552177,17.2511,17.2807,17.3071,17.0929,17.232950,-0.868760
6,AAPL,2013-10-08,289041924,17.1764,17.4979,17.5229,17.1621,17.339825,-0.524774
8,AAPL,2013-10-10,276745897,17.4871,17.5471,17.5850,17.3943,17.503375,1.191371
...,...,...,...,...,...,...,...,...,...
2506,AAPL,2023-09-14,60895760,175.7400,174.0000,176.1000,173.5800,174.855000,-0.367521
2508,AAPL,2023-09-18,67257570,177.9700,176.4800,179.3800,176.1700,177.500000,1.167703
2510,AAPL,2023-09-20,58436180,175.4900,179.2600,179.6950,175.4000,177.461250,-0.491344
2512,AAPL,2023-09-22,56725390,174.7900,174.6700,177.0790,174.0500,175.147250,0.278971


In [ ]:
def subset_dates(data, freq):

    date_index = pd.date_range(start=aapl.loc[0, 'Date'], end=aapl.loc[len(aapl)-1 ,'Date'], freq=freq)
    
    orig_dates = data['Date']

    orig_dates = np.asarray(orig_dates)
    date_index = np.asarray(date_index)

    dates_intersect = np.intersect1d(orig_dates, date_index)

    date_mask = np.isin(orig_dates, dates_intersect)

    subset_of_dates = aapl[date_mask]
    
    return subset_of_dates

So, there is a problem with this dataset that sorta invalidates this method. Now we can definitely proceed further in this direction (possibly providing no degree of consistency in date selection), or we can throw something together a little cheaper, but more consistent

Since we converted the `'Date'` column from a `str` to a `DateTime` type, we can access various pieces of information about it, and even create our own subsections based on that information.

Here's why I am taking this idea as an approach:
- The real historical stock market data does not have a date for every single day through it's 10 years of data. This is because the stock market is not "open" 7 days a week, 52 weeks a year. And there are probably holidays that it is closed on as well. Oh well, so be it.
- We can exploit the `DateTime` datatype to actually subset dates based on the week day the date falls in. It might be a little more *jank* or a little bit extra effort, but we can ourselves create a thorough subsectioning of this data.

So now it becomes data-cleaning and pre-processing even further! WOO! This is a big part of working with data in any sort of Data Science - Machine Learning field, so it's best to get used to making these kinds of decisions now than later.

Now, here is my idea:
1. Using index step-sizes to step through on a day level
2. Subset weeks by 'Monday'; In essence:
    1. 'Highlight' Monday
    2. Take that day, and the following Friday, and concat as a week

# This whole thing gives me a headache

So I am going to step away from this task for a short bit. It is not directly useful in this moment, and doesn't get me closer to the end goal of simulating a stock market, so I am going to elect to set this task to the side and continue the other work in a new journal. 

I do intend to return to it however, at a later date.